# 4. Modelagem

## 4.1 Linhagem dos dados

**Data Lineage da tabela de Guias do ITBI, camada bronze:**

As tabelas base foram lidas diretamente das URLs disponíveis no site da prefeitura de São Paulo, conforme descrito na seção 3.1.1.

O detalhamento das colunas foi omitido, por serem as mesmas colunas dentre as 4 tabelas base (com exceção do nome da penúltima coluna, que continha variações, e foi corrigido no processamento da tabela agregada, item 3.2.1).

A produção das 4 tabelas base e da tabela agregada de Guias do ITBI da camada bronze é assunto da sessão 3.2.1, onde todos os processos de transformação necessários foram descritos. 

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Lineage_Bronze_ITBI)



**Data Lineage da tabela de CEP, camada bronze:**

Os arquivos base foram gravados manualmente no schema staging, a partir de arquivos zip disponíveis no site CEP Aberto (conforme documentado na seção 3.1.2).

A produção da tabela agregada de CEP da camada bronze é assunto da sessão 3.2.3, onde todos os processos de transformação necessários foram descritos.

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Lineage_Bronze_CEP)

**Data Lineage das camadas prata e ouro:**

A produção das tabelas tratadas da camada prata é assunto da sessão 6, onde todos os processos de transformação foram descritos. 
A tabela de Guias de ITBI foi assunto do item 6.1, enquanto a tabela de CEP foi assunto o item 6.2.

A formação da tabela flat da camada ouro, por sua vez, foi descrita na sessão 7.

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Lineage_Prata_Ouro)

## 4.2 Modelo de dados

**Camada Prata:**

O modelo de dados construído na camada prata foi um modelo estrela, em que a tabela "guias_itbi_prata" ocupa a função de tabela fato.

Já as tabelas "cep_sp_capital" e "tabela_dim_usos" são as tabelas dimensão, conforme representado no relacionamento abaixo: 

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Modelo_Prata_Estrela)

**Camada Ouro:**

Já na camada ouro, o modelo de dados é flat, composto por uma única tabela agregada "guias_itbi_ouro".

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Modelo_Ouro_Flat)


## 4.3 Catálogo de dados

**Catálogo de dados da tabela "guias_itbi_prata":**

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Dicionario_ITBI_1)
![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Dicionario_ITBI_2)

**Catálogo de dados da tabela "cep_sp_capital":**

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Dicionario_CEP)

**Catálogo de dados da tabela "tabela_dim_usos":**

![](/Workspace/Users/izabelagreg@gmail.com/mvp-puc-eng-dados/Figuras/Dicionario_Usos)

# 5. Qualidade dos dados

A análise da qualidade dos dados focou apenas nas colunas relevantes para a análise proposta, que serão selecionadas a partir da camada prata:
- Número do cadastro (SQL)
- CEP
- Valor da Transação (declarado pelo contribuinte)
- Data da Transação
- Proporção Transmitida
- Tipo de Financiamento
- Valor Financiado
- Matrícula do imóvel
- Área do Terreno (m2)
- Área Construída (m2)
- Uso (IPTU)

A verificação de qualidade foi organizada em 5 etapas:

1. **Unicidade**: 
- Como não existe chave primária na base bruta (camada bronze), e um mesmo imóvel pode ser transacionado em diversas partes iguais, no mesmo dia, podemos encontrar linhas equivalentes na base de dados. A tabela bronze não tem, por regra de negócio, garantia de unicidade de suas linhas. Na camada prata iremos criar uma chave primária surrogate para que cada linha seja única.

2. **Integridade**:
- Verificar se existem valores nulos nas colunas
- Importante notar que as colunas "Tipo de Financiamento" e "Valor Financiado" são campos não obrigatórios nos dados raw, logo podem ter valores não preenchidos.

3. **Formato/ padrão/ tipo de dado:**
- Tipo de dados: Na criação das tabelas consolidadas na camada bronze, o tipo de dado de cada coluna foi especificado. Todos os registros que não poderiam ser convertidos a esse tipo, assumiram valor NULL e serão verificados na etapa de integridade.
- Verificar se os percentuais da coluna "Proporção Transmitida" estão escritos no formato esperado para um percentual

4. **Validade:**
- Verificar se valores financeiros (Valor da transação e Valor financiado) e áreas (Área do terreno e Área construída) são sempre maiores ou iguais a zero
- Verificar se o Valor declarado da Transação é diferente de zero
- Verificar se existe Data de Transação antes ou depois do período em estudo (jan-22 a out-25)
- Verificar se existem valores de área muito altos que poderiam indicar erros de preenchimento 
- Verificar se a Proporção Transmitida é maior que zero e menor que 100%

5. **Consistência:**
- Verificar se os Tipos de Financiamento são valores de uma lista esperada
- O valor de Área Construída pode ser nulo, se e somente se, o tipo de imóvel for terreno
- Verificar se todos os registros que têm Valor Financiado zero, correpondem a registros que não têm informação na coluna Tipo de Financiamento 
- Verificar se todos os registros cujo Tipo de Financiamento é 'nan' apresentam Valor Financiado zerado
- O Valor Financiado não deve ser maior que o Valor da Transação
- Validar se todos os valores de CEP da tabela de ITBI aparecem na tabela de CEPs dos correios (CEP aberto)
- Verificar se os códigos de uso da tabela de ITBI estão na lista de uso do IPTU

Os testes descritos acima estão representados em código abaixo. Os problemas de dados identificados serão resolvidos nas transformações para criar a camada prata.
 



## 5.1 Integridade:

In [0]:
%sql
/* 2. Integridade */
/* Verificar se existem valores nulos nas colunas */
/* Como são poucos os registros nulos, eles podem ser deletados da base sem alterar significativamente a análise */

SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN No_do_Cadastro__SQL_ IS NULL THEN 1 ELSE 0 END) AS null_No_do_Cadastro__SQL_,
  SUM(CASE WHEN CEP IS NULL THEN 1 ELSE 0 END) AS null_CEP,
  SUM(CASE WHEN `Valor_de_Transação__declarado_pelo_contribuinte_` IS NULL THEN 1 ELSE 0 END) AS null_Valor_de_Transacao,
  SUM(CASE WHEN `Data_de_Transação` IS NULL THEN 1 ELSE 0 END) AS null_Data_de_Transacao,
  SUM(CASE WHEN `Proporção_Transmitida__pct_` IS NULL THEN 1 ELSE 0 END) AS null_Proporcao_Transmitida,
  SUM(CASE WHEN `Matrícula_do_Imóvel` IS NULL THEN 1 ELSE 0 END) AS null_Matricula_do_Imovel,
  SUM(CASE WHEN `Área_do_Terreno__m2_` IS NULL THEN 1 ELSE 0 END) AS null_Area_do_Terreno__m2_,
  SUM(CASE WHEN `Área_Construída__m2_` IS NULL THEN 1 ELSE 0 END) AS null_Area_Construida__m2_,
  SUM(CASE WHEN Uso__IPTU_ IS NULL THEN 1 ELSE 0 END) AS null_Uso__IPTU_
FROM mvp_engdados_puc.bronze.guias_itbi_bronze


## 5.2 Formato/ padrão/ tipo de dado:

Conforme demonstrado abaixo, a coluna "Proporção Transmitida" está despadronizada em seu preenchimento.

O valor de 50%, por exemplo, seria escrito como 50 ou 0.50.

Na etapa de formação da camada prata iremos tratar esta coluna da seguinte maneira:
- Valores menores que 1 serão multiplicados por 100
- Valores maiores que 1 serão considerados no formato correto

In [0]:
%sql
/* Veririficar se os percentuais da coluna "Proporção Transmitida" estão escritos no formato esperado para um percentual */
SELECT 
`Proporção_Transmitida__pct_`,
COUNT(*)
  
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

GROUP BY `Proporção_Transmitida__pct_`
ORDER BY COUNT(*) DESC



## 5.3 Validade:

In [0]:
%sql
/* Verificar se valores financeiros e áreas são sempre maiores ou iguais a zero */
/* Verificar se o valor da transação é maior que zero */

SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN `Valor_de_Transação__declarado_pelo_contribuinte_` <= 0 THEN 1 ELSE 0 END) AS transacao_negativa_nula,
  SUM(CASE WHEN Valor_Financiado < 0 THEN 1 ELSE 0 END) AS financiamento_negativo,
  SUM(CASE WHEN `Área_do_Terreno__m2_` < 0 THEN 1 ELSE 0 END) AS area_terreno_negativo,
  SUM(CASE WHEN `Área_Construída__m2_` < 0 THEN 1 ELSE 0 END) AS area_construida_negativa

FROM mvp_engdados_puc.bronze.guias_itbi_bronze
WHERE CEP IS NOT NULL /* Retira linhas que já sabemos que estão corrompidas */


In [0]:
%sql
/* Verificar se existem datas antes ou depois do período que estamos trabalhando jan-2022 a out-2025 */

SELECT COUNT(*) AS qtd_fora_janela
FROM mvp_engdados_puc.bronze.guias_itbi_bronze
WHERE `Data_de_Transação` >= DATE('2025-11-01') OR `Data_de_Transação` < DATE('2022-01-01')


De pesquisas na internet, temos que a propriedade com maior tamanho na cidade de São Paulo tem cerca de 22.000 metros quadrados de terreno, e 11.000 metros quadrados de área construída. 

Considerando estes valores como referência (com alguma margem de erro), iremos considerar muito grande qualquer área de terreno acima de 25.000 metros quadrados, e área construída acima de 12.000 metros quadrados.

In [0]:
%sql
/* Verificar se existem registros com área construída muito grande */

SELECT COUNT(*) as qtd

FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE `Área_Construída__m2_` >= 12000

In [0]:
%sql
/* Verificar se existem registros com área do terreno muito grande */

SELECT COUNT(*) as qtd

FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE `Área_do_Terreno__m2_` >= 25000

In [0]:
%sql
/* Verificar se a "Proporção Transmitida" é menor que 100% */
SELECT COUNT(*) as qtd
  
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE `Proporção_Transmitida__pct_` > 100

In [0]:
%sql
/* Verificar se a "Proporção Transmitida" é maior que zero */

SELECT COUNT(*) as qtd
  
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE `Proporção_Transmitida__pct_` <= 0


## 5.4 Consistência:

In [0]:
%sql
/* Verificar se os tipos de financiamento são valores de uma lista esperada */

SELECT Tipo_de_financiamento, COUNT(*) AS qtd
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE CEP IS NOT NULL /* Retira as linhas que já sabemos que estão corrompidas */

GROUP BY Tipo_de_financiamento
ORDER BY qtd DESC


In [0]:
%sql
/* Verificar se, quando a área construída é zero, se trata de um terreno */

SELECT
  `Descrição_do_uso__IPTU_` AS descricao_uso,
  COUNT(*) AS qtd
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE `Área_Construída__m2_` = 0

GROUP BY `Descrição_do_uso__IPTU_`
ORDER BY qtd DESC

In [0]:
%sql
/* Verificar se todos os registros que tem valor financiado zero correspondem a registros que não tem informação na coluna "Tipo de financiamento" */

SELECT
  Tipo_de_financiamento,
  COUNT(*) AS total_financiamento_zero
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE Valor_Financiado = 0 
GROUP BY Tipo_de_financiamento


In [0]:
%sql
/* Verificar se todos os registros cujo tipo de financiamento é nan apresentam valor de financiamento zerado */

SELECT
  COUNT(CASE WHEN Valor_Financiado <> 0 THEN 1 END) AS total_financiamento_nao_nulo
FROM mvp_engdados_puc.bronze.guias_itbi_bronze
WHERE Tipo_de_Financiamento = 'nan' 


In [0]:
%sql
/* O valor financiado não deve ser maior que o valor da transação. Esse comportamento indica erro de preenchimento em algum desses valores. */

SELECT
  No_do_Cadastro__SQL_,
  `Valor_de_Transação__declarado_pelo_contribuinte_`,
  Valor_Financiado
FROM mvp_engdados_puc.bronze.guias_itbi_bronze
WHERE Valor_Financiado > `Valor_de_Transação__declarado_pelo_contribuinte_`


In [0]:
%sql
/* Validar se todos os valores de CEP da tabela bronze do ITBI aparecem na tabela de CEPs dos correios */

SELECT
  CASE WHEN p.cep IS NULL THEN 'NÃO ENCONTRADO' ELSE 'ENCONTRADO' END AS status_cep,
  COUNT(b.cep) AS qtd

FROM mvp_engdados_puc.bronze.guias_itbi_bronze b

LEFT JOIN mvp_engdados_puc.prata.cep_sp_capital p
ON b.cep = p.cep

GROUP BY status_cep

In [0]:
%sql
/* Verificar se os códigos de uso estão na lista de uso do iptu */

SELECT
  CASE WHEN p.uso IS NULL THEN 'NÃO ENCONTRADO' ELSE 'ENCONTRADO' END AS status_uso,
  COUNT(b.Uso__IPTU_) AS qtd

FROM mvp_engdados_puc.bronze.guias_itbi_bronze b

LEFT JOIN mvp_engdados_puc.bronze.tabela_dim_usos p
ON b.Uso__IPTU_ = p.uso

GROUP BY status_uso

## 5.5 Conclusões de qualidade:

- Existem valores nulos nas colunas em quantidade irrisória, logo podem ser deletados na base tratada
- Eliminando os registros NULL, todos os registros estarão com o tipo de dado correto
- A coluna "Proporção Transmitida" está despadronizada no formato dos valores percentuais. Iremos equalizar no tratamento da cama prata.
- Existem registros com "Valor de Transação" nulo, e "Valor Financiado" negativo. Como são poucas ocorrências, podemos eliminar na base tratada.
- Existem registros com "Data da Transação" fora do período de interesse (jan-22 a out-25). Por não serem objeto deste estudo, iremos descartar estes registros.
- Existem registros com área construída ou de terreno que podemos considerar muito grandes (acima de 12.000 e 25.000 metros quadrados respectivamente). Essa característica indica erro de preenchimento, logo iremos descartar estas linhas.
- Existem transações em que a "Proporção Transmitida" era nula ou maior que 100%. Como esta informação corrompida poderia impactar diretamente na análise, iremos descartar essas transações. Felizmente a ocorrência deste erro é pequena.
- Os tipos de financiamento que aparecem na base estão dentro do esperado
- Valores de área construída nulos só aparecem para terrenos, conforme esperado
- Em alguns poucos registros com "Valor Financiado" nulo, aparece o "Tipo de Financiamento". Por essa inconsistência serão desconsiderados. 
- Existem registros em que o "Valor Financiado" é não nulo, para financiamento do tipo 'nan'. Vamos forçar que este valor seja zero, conforme esperado.
- Existem registros em que o "Valor Financiado" é maior que o "Valor da Transação". Como esta característica indica erro de preenchimento, e a ocorrência não é tão frequente que possa impactar a análise, iremos descartar estas linhas.
- Existem alguns CEPs da base do ITBI que não aparecem na base do site CEP Aberto. Alguns são CEPs incorretos, outros existem demonstrando que a base do CEP Aberto está incompleta. Como esta ocorrência é insignificante perante o volume total de registros, por simplicidade, iremos desconsiderar os CEPs que não aparecem na base do site CEP Aberto.
- Existem registros que contêm códigos de uso inexistentes. Como essa ocorrência é pouco frequente, iremos eliminar da base.



# 6. Transformação dos dados e formação da camada prata

## 6.1 Tabela tratada de guias do ITBI (camada prata):

A tabela "guias_itbi_prata" é o principal dataset estruturado do projeto. Ela contém os dados tratados e limpos das transações imobiliárias de compra e venda que geraram o Imposto de Transmissão de Bens Imóveis (ITBI) na cidade de São Paulo, cobrindo o período de 2022 a outubro de 2025.

Esta tabela foi criada a partir da camada Bronze, após a aplicação de regras de qualidade incluindo:

- Padronização de tipos de dados
- Eliminação de valores inválidos ou inconsistentes para cada coluna
- Geração de uma chave primária surrogate para garantir a unicidade de cada linha

Passo a passo do tratamento dos dados, para a formação da camada prata:

**PASSO 1 - Limpeza da base:**

- Criar uma chave primária surrogate para que cada linha seja única
- Selecionar apenas as colunas de interesse para a análise em questão
- Renomear colunas para nomes mais amigáveis
- Padronizar o formato dos percentuais da coluna "Proporção transmitida" (valores menores que 1 serão multiplicados por 100)
- Forçar que todos os registros de financiamento do tipo 'nan' apareçam com valor financiado zero
- Adicionar uma coluna que representa o preço por metro quadrado de cada transação
- Filtrar apenas transações de compra e venda (90% dos registros), uma vez que outros tipos de operação podem ter preços fora de mercado
- Excluir linhas com valores NULL (exceto para tipo do financiamento, que pode não existir) 
- Excluir registros que tenham "Valor de Transação" ou "Valor Financiado" negativos
- Excluir registros que tenham "Valor de Transação" nulo
- Excluir registros com área construída ou área do terreno anormalmente grandes (acima de 12000 e 25000 m2 respectivamente)
- Excluir registros com "Proporção Transmitida" nula ou acima de 100%
- Excluir registros em que o "Valor financiado" seja maior que o "Valor da Transação"
- Filtrar apenas transações que ocorreram na janela de interesse (jan-22 a out-25). Como a base representa as guias de impostos pagas a partir de 2022, algumas transações podem ter ocorrido antes desse período, com pagamento em ano posterior.
- Eliminar registros com "Valor Financiado" nulo e "Tipo de Financiamento" diferente de 'nan'
- Eliminar registros com CEP que não aparece na base do CEP Aberto
- Eliminar registros com código de uso que não aparece a base de Usos do IPTU


**PASSO 2 - Adicionar variável de interesse para a análise:**
- Adicionar a coluna de preço por metro quadrado de cada transação a ser calculada como o valor da transação dividido pela área construída que foi efetivamente transmitida. 
Em outras palavras: "Valor da Transação" / ("Área Construída" * "Proporção Transmitida" / 100)

In [0]:
%sql
/* Criacão da tabela tratada de guias de ITBI na camada prata */

CREATE OR REPLACE TABLE mvp_engdados_puc.prata.guias_itbi_prata AS

SELECT *, 
 try_divide(Valor_Transacao, Area_Construida * Proporcao_Transmitida / 100) AS preco_m2

FROM 
(
SELECT 
  monotonically_increasing_id() AS primary_key, -- coluna de chave primária
  No_do_Cadastro__SQL_ AS No_Cadastro,
  CEP,
  `Valor_de_Transação__declarado_pelo_contribuinte_` AS Valor_Transacao,
  `Data_de_Transação` AS Data_Transacao,
  CASE 
    WHEN `Proporção_Transmitida__pct_` <= 1 THEN `Proporção_Transmitida__pct_` * 100
    ELSE `Proporção_Transmitida__pct_`
  END AS Proporcao_Transmitida,
  Tipo_de_financiamento AS Tipo_Financiamento,
  CASE 
    WHEN Tipo_de_financiamento = 'nan' THEN 0
    ELSE Valor_Financiado
  END AS Valor_Financiado,
  `Matrícula_do_Imóvel` AS Matricula_Imovel,
  `Área_do_Terreno__m2_` AS Area_Terreno,
  `Área_Construída__m2_` AS Area_Construida,
  Uso__IPTU_ AS Uso_IPTU
  
FROM mvp_engdados_puc.bronze.guias_itbi_bronze

WHERE 1=1 
/* Total de linhas da base bronze: 745.179 */

/* Essa condição de negócio foi responsável por excluir 74.068 registros (~10% da base) total 671.111 registros */
AND `Natureza_de_Transação` IN ('1.Compra e venda')

/* Esse bloco de condições NOT NULL foi responsável por eliminar apenas mais 1 linha 671.110 */
AND No_do_Cadastro__SQL_ IS NOT NULL
AND CEP IS NOT NULL
AND `Valor_de_Transação__declarado_pelo_contribuinte_` IS NOT NULL
AND `Data_de_Transação` IS NOT NULL
AND `Proporção_Transmitida__pct_` IS NOT NULL
AND `Matrícula_do_Imóvel` IS NOT NULL
AND `Área_do_Terreno__m2_` IS NOT NULL
AND `Área_Construída__m2_` IS NOT NULL
AND Uso__IPTU_ IS NOT NULL

/* Esse segundo bloco de condições foi responsável por eliminar mais 43.113 registros (~6% da base), total 627.997 */
AND `Valor_de_Transação__declarado_pelo_contribuinte_` > 0
AND Valor_Financiado >= 0
AND `Área_Construída__m2_` <= 12000
AND `Área_do_Terreno__m2_` <= 25000
AND `Proporção_Transmitida__pct_` > 0
AND `Proporção_Transmitida__pct_` <= 100
AND `Valor_de_Transação__declarado_pelo_contribuinte_` >= Valor_Financiado

/* Esse filtro da análise foi responsável por eliminar mais 2.732 registros (percentual irrisório), total 625.265 */
AND `Data_de_Transação` >= DATE('2022-01-01')
AND `Data_de_Transação` < DATE('2025-11-01')

/* Essa correção dos tipos de financiamento eliminou mais 4 linhas (número irrisório), total 625.261 */
AND (Valor_Financiado <> 0 OR (Valor_Financiado = 0 AND Tipo_de_financiamento = 'nan'))

/* Essa restrição de chave estrangeira eliminou mais 261 linhas (número irrisório), total 625.000 */
AND CEP IN (SELECT CEP FROM mvp_engdados_puc.prata.cep_sp_capital)

/* Essa restrição de chave estrangeira eliminou mais 1.818 linhas (percentual irrisório), total 623.182 */
AND Uso__IPTU_ IN (SELECT Uso FROM mvp_engdados_puc.bronze.tabela_dim_usos)
)

In [0]:
%sql
/* Visualização da tabela criada */

SELECT * FROM mvp_engdados_puc.prata.guias_itbi_prata LIMIT 1000

## 6.2 Tabela de CEPs dos correios (CEP Aberto):

Esta tabela dimensão compõe o esquema estrela da camada prata e serve para informar o bairro a que cada CEP da cidade de São Paulo se refere.

In [0]:
%sql
/* Criação da tabela de CEPs da cidade de São Paulo na camada prata */
/* As análises posteriores serão feitas na granularidade bairro, logo precisamos apenas dessa coluna, além da chave */

CREATE OR REPLACE TABLE mvp_engdados_puc.prata.cep_sp_capital AS 

SELECT DISTINCT
  CAST(cep AS BIGINT) AS cep, 
  bairro

FROM mvp_engdados_puc.bronze.cepaberto_sp

WHERE cidade IN 
  (
    SELECT codigo_cidade
    FROM mvp_engdados_puc.bronze.cepaberto_cidades
    WHERE nome_cidade = 'São Paulo'
  )

In [0]:
%sql
/* Visualizacão da tabela */
SELECT * FROM mvp_engdados_puc.prata.cep_sp_capital LIMIT 100

## 6.3 Tabela de Usos segundo IPTU:

Esta tabela dimensão compõe o esquema estrela da camada prata e serve como de-para para a lista de identificadores das categorias de uso do imóvel, segundo o cadastro do IPTU. A descrição explicita a finalidade preponderante, referente a determinado código.

In [0]:
%sql
/* Tabela de usos segundo IPTU já estava tratada na camada bronze, logo será apenas copiada para a cama prata */

CREATE OR REPLACE TABLE mvp_engdados_puc.prata.tabela_dim_usos AS
SELECT 
  Uso,
  `Descrição` AS descricao
FROM mvp_engdados_puc.bronze.tabela_dim_usos;


In [0]:
%sql
/* Visualização da tabela */
SELECT *
FROM mvp_engdados_puc.prata.tabela_dim_usos;

# 7. Transformações de dados e formação da camada ouro

A camada Ouro (guias_itbi_ouro) representa a visão final e agregada do projeto, otimizada especificamente para o problema de negócio (estratégia de House Flipping).
Esta tabela é o resultado da agregação de dados da camada Prata, onde a granularidade foi alterada para o nível de bairro e mês.

O modelo é Flat (desnormalizado) e serve como fonte direta para os gráficos e insights da seção de análise final, simplificando as consultas.

Segue o detalhamento das transformações realizadas:
- Selecionar apenas as colunas mais relevantes para o resultado de negócio final 
- Agrupar os registros por mês e por bairro
- Contar o número de transações em cada grupo criado
- Somar o valor total transacionado em cada grupo criado
- Calcular a média da variável de interesse (Preço do metro quadrado) para cada grupo
- Enriquecer a tabela Fato com as tabelas Dimensão do esquema estrela da camada prata, para formar um única tabela enriquecida flat
- Filtrar apenas as transações em que 100% do imóvel foi adquirido, uma vez que transacionar apenas parte do imóvel pode alterar seu valor perante o mercado
- Filtrar apenas os casos de Uso do IPTU que correspondem ao uso residencial (Residência, Residência coletiva, Apartamento em condomínio, Flat residencial em condomínio), excluindo campras de prédio inteiro


In [0]:
%sql
/* Criacão da tabela flat na camada ouro */

CREATE OR REPLACE TABLE mvp_engdados_puc.ouro.guias_itbi_ouro AS

SELECT 
  CAST(DATE_TRUNC('MONTH', a.Data_Transacao) AS DATE) AS mes_transacao,
  b.bairro,
  COUNT(*) AS num_transacoes,
  ROUND(SUM(a.Valor_Transacao), 2) AS valor_transacionado,
  ROUND(AVG(a.preco_m2), 2) AS avg_preco_m2
  
FROM mvp_engdados_puc.prata.guias_itbi_prata AS a

LEFT JOIN mvp_engdados_puc.prata.cep_sp_capital AS b
ON a.CEP = b.CEP

WHERE 1=1 
AND Proporcao_Transmitida = 100 /* Considera apenas imóveis vendidos por inteiro */
AND Uso_IPTU IN /* Filtra apenas imóveis de uso residencial */
  (
  SELECT Uso 
  FROM mvp_engdados_puc.prata.tabela_dim_usos 
  WHERE descricao IN ('RESIDÊNCIA', 'RESIDÊNCIA COLETIVA, EXCLUSIVE CORTIÇO (MAIS DE UMA RESIDÊNCIA NO LOTE)', 'APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)', 'FLAT RESIDENCIAL EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)')
  )

GROUP BY mes_transacao, bairro

In [0]:
%sql
/* Visualização da tabela criada */

SELECT *
FROM mvp_engdados_puc.ouro.guias_itbi_ouro 
WHERE mes_transacao = '2025-10-01'
ORDER BY avg_preco_m2 DESC
